<a href="https://colab.research.google.com/github/severus-fan/Machine_Learning/blob/main/PM2_5_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# OpenCV 套件，和 Google Drive 無關
import cv2
# import Google Drive 套件
from google.colab import drive
# 將自己的雲端硬碟掛載上去
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import sys
import pandas as pd
import numpy as np
data = pd.read_csv('/content/gdrive/MyDrive/ML2020/hw1_regression/dataset/train.csv', encoding = 'big5')
data[data == 'NR'] = 0
data = data.iloc[:, 3:]

In [3]:
raw_data = data.to_numpy()
raw_data

array([['14', '14', '14', ..., '15', '15', '15'],
       ['1.8', '1.8', '1.8', ..., '1.8', '1.8', '1.8'],
       ['0.51', '0.41', '0.39', ..., '0.35', '0.36', '0.32'],
       ...,
       ['36', '55', '72', ..., '118', '100', '105'],
       ['1.9', '2.4', '1.9', ..., '1.5', '2', '2'],
       ['0.7', '0.8', '1.8', ..., '1.6', '1.8', '2']], dtype=object)